In [1]:
from collections import defaultdict
from dataclasses import dataclass
import os
import random
import time
from typing import Callable, Dict, List, Generator, Tuple
from data_pre_process import *

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

import torch
from torch import nn, optim
import torch.cuda.amp
from pathlib import Path

from torch.utils.data import Dataset, Subset, DataLoader

from transformers import BertTokenizer, AdamW, BertModel, get_linear_schedule_with_warmup, BertPreTrainedModel

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  if sys.path[0] == '':


In [2]:
data_dir_t = Path('data_2/v1.0/train')
data_path_t = data_dir_t/'nq-train-00.jsonl.gz'

data_dir_v = Path('data_2/v1.0/dev')
data_path_v = data_dir_v/'nq-dev-00.jsonl.gz'

In [3]:
chunksize = 200
max_seq_len = 384
max_question_len = 64
doc_stride = 128
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case='uncased' in 'bert-base-uncased')

convert_func = functools.partial(convert_data,
                                 tokenizer=tokenizer,
                                 max_seq_len=max_seq_len,
                                 max_question_len=max_question_len,
                                 doc_stride=doc_stride)

In [4]:
start = time.time()
with gzip.open(data_path_t, "rb") as f:
    data = f.read()
x = data.splitlines()
data_reader = JsonlReader(x, convert_func, chunksize=chunksize)
end = time.time()
print("Loading Data:", end - start, "seconds")

Loading Data: 37.94343280792236 seconds
